<a href="https://colab.research.google.com/github/CalculatedContent/xgboost2ww/blob/main/notebooks/W8GoodModelsXGBoost2WW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# xgboost2ww Experiment (100 random models)

This starter notebook trains 100 random XGBoost models and evaluates the result.  

It shows you how to:
1. Pick a “good” XGBoost model using **training-only cross-validation**.
2. Evaluate it on a **true holdout test set** (never used in CV or OOF).
3. Use **xgboost2ww.convert()** to build tiny PyTorch layers for the **W** (W7 default)
4. Run **WeightWatcher** to estimate:
   - **α (alpha)**: heavy-tail exponent estimate
   - **traps**: randomization spikes proxy (WeightWatcher diagnostic)

**Note:** For an initial evaluation, you do **not** need `detX=True` in WeightWatcher.


<hr>

### User can set:

Matrix = "W7"

TARGET_DATASETS = 100

In [40]:

# Pick Matrix W1 | W2 | W7 | W8
MATRIX = "W8"

# Starter run size
TARGET_DATASETS = 100



# Set up folder on Google Drive to save final results[link text](https://)

In [41]:

from google.colab import drive
import os
import pandas as pd
from datetime import datetime

# Mount Drive
drive.mount("/content/drive", force_remount=False)

# Create output folder
GDRIVE_DIR = "/content/drive/MyDrive/xgboost2ww_runs"
os.makedirs(GDRIVE_DIR, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Install dependencies

In [42]:
!pip -q install "pandas==2.2.2" xgboost weightwatcher scikit-learn openml scipy pyarrow
!apt-get -qq update && apt-get -qq install -y git

# Clone + install xgboost2ww (public repo)
!pip install xgboost2ww

import xgboost2ww
print("xgboost2ww loaded from:", getattr(xgboost2ww, "__file__", None))

from xgboost2ww import convert
print("Imported xgboost2ww.convert OK")

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
xgboost2ww loaded from: /usr/local/lib/python3.12/dist-packages/xgboost2ww/__init__.py
Imported xgboost2ww.convert OK


## Imports and settings

We’ll run a small starter experiment over a handful of OpenML binary datasets.

Key settings:
- `TARGET_DATASETS`: how many datasets to run (starter: 10)
- `NFOLDS`: OOF folds for xgboost2ww matrices
- `T_TRAJ`: how many trajectory points along boosting to sample

In [43]:
import warnings, time
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import xgboost as xgb
import openml

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

import torch
import weightwatcher as ww

# Reproducibility
RNG = 0
rng = np.random.default_rng(RNG)


# Train/test split (true holdout)
TEST_SIZE = 0.20

# OOF matrix construction
NFOLDS = 5
T_TRAJ = 160

# Data cap + safety guard
MAX_OPENML_ROWS = 60000
MAX_FEATURES_GUARD = 50_000

# “Good model” selection (training-only CV)
GOOD_TRIALS = 5
CV_MAX_ROUNDS = 3000
CV_EARLY_STOP = 150
MIN_GOOD_TEST_ACC = 0.75

# OpenML suites to pull datasets from
SUITE_IDS = [14, 99, 225]

## Optional: GPU detection for XGBoost

If a GPU is available, we’ll use XGBoost’s `gpu_hist` for faster training.
If not, we fall back to CPU `hist`.

In [44]:
def xgb_gpu_available() -> bool:
    try:
        Xtmp = np.random.randn(256, 8).astype(np.float32)
        ytmp = (Xtmp[:, 0] > 0).astype(np.int32)
        dtmp = xgb.DMatrix(Xtmp, label=ytmp)
        params = dict(
            objective="binary:logistic",
            eval_metric="logloss",
            tree_method="gpu_hist",
            predictor="gpu_predictor",
            max_depth=2,
            learning_rate=0.2,
            seed=RNG,
        )
        _ = xgb.train(params=params, dtrain=dtmp, num_boost_round=5, verbose_eval=False)
        return True
    except Exception:
        return False

USE_GPU = xgb_gpu_available()
print("XGBoost GPU available:", USE_GPU)

XGBoost GPU available: False




```
## Load OpenML binary datasets (with preprocessing)

We:
- Keep only **binary** classification datasets
- One-hot encode categorical features
- Impute missing values
- Optionally cap dataset size (`MAX_OPENML_ROWS`) for speed
```



In [45]:
def factorize_binary(y_raw):
    y_codes, uniques = pd.factorize(y_raw)
    if len(uniques) != 2:
        return None
    return y_codes.astype(int)

def make_preprocessor(Xdf: pd.DataFrame):
    cat_cols = Xdf.select_dtypes(include=["object", "category", "bool"]).columns.tolist()
    num_cols = [c for c in Xdf.columns if c not in cat_cols]
    transformers = []
    if len(num_cols):
        transformers.append(("num", Pipeline([("imp", SimpleImputer(strategy="median"))]), num_cols))
    if len(cat_cols):
        transformers.append(("cat", Pipeline([
            ("imp", SimpleImputer(strategy="most_frequent")),
            ("oh", OneHotEncoder(handle_unknown="ignore", sparse_output=True))
        ]), cat_cols))
    if not transformers:
        raise ValueError("no usable columns")
    return ColumnTransformer(transformers=transformers, remainder="drop", sparse_threshold=0.3)

def enumerate_openml_dataset_ids_from_suites(suite_ids):
    ids, seen = [], set()
    for sid in suite_ids:
        suite = openml.study.get_suite(sid)
        for did in suite.data:
            did = int(did)
            if did not in seen:
                ids.append(did)
                seen.add(did)
    return ids

def load_openml_dataset_by_id(did: int):
    ds = openml.datasets.get_dataset(did)
    target = ds.default_target_attribute
    Xdf, y_raw, _, _ = ds.get_data(dataset_format="dataframe", target=target)

    y = factorize_binary(y_raw)
    if y is None:
        return None

    if MAX_OPENML_ROWS is not None and len(Xdf) > MAX_OPENML_ROWS:
        take = rng.choice(len(Xdf), size=MAX_OPENML_ROWS, replace=False)
        Xdf = Xdf.iloc[take].reset_index(drop=True)
        y = y[take]

    pre = make_preprocessor(Xdf)
    X = pre.fit_transform(Xdf)
    return X, y.astype(int), ds.name, did

## Pick a “good” XGBoost model using training-only CV

For each dataset:
1. Split into **train/test**
2. Run CV on the **train only** set to pick hyperparameters + early-stopping rounds
3. Train a final model on train
4. Evaluate once on the holdout test set

We keep only datasets where the holdout test accuracy is at least `MIN_GOOD_TEST_ACC`.

In [46]:
def pick_good_params_via_cv(Xtr, ytr, nfold=5, *, dataset_id: int):
    dtrain = xgb.DMatrix(Xtr, label=ytr)
    local_rng = np.random.default_rng(RNG + int(dataset_id))  # stable across runs

    best = None
    best_score = np.inf

    for _ in range(GOOD_TRIALS):
        params = dict(
            objective="binary:logistic",
            eval_metric="logloss",
            tree_method="hist",
            seed=RNG,
            learning_rate=float(10 ** local_rng.uniform(-2.0, -0.6)),   # 0.01..0.25
            max_depth=int(local_rng.integers(2, 7)),
            min_child_weight=float(10 ** local_rng.uniform(0.0, 2.0)),  # 1..100
            subsample=float(local_rng.uniform(0.6, 0.9)),
            colsample_bytree=float(local_rng.uniform(0.6, 0.9)),
            reg_lambda=float(10 ** local_rng.uniform(0.0, 2.0)),        # 1..100
            gamma=float(local_rng.uniform(0.0, 0.5)),
        )
        if USE_GPU:
            params["tree_method"] = "gpu_hist"
            params["predictor"] = "gpu_predictor"

        cv = xgb.cv(
            params=params,
            dtrain=dtrain,
            num_boost_round=CV_MAX_ROUNDS,
            nfold=nfold,
            stratified=True,
            early_stopping_rounds=CV_EARLY_STOP,
            seed=RNG,
            verbose_eval=False,
        )

        score = float(cv["test-logloss-mean"].iloc[-1])
        rounds = int(len(cv))
        if score < best_score:
            best_score = score
            best = (params, rounds, score)

    return best  # (params, rounds, cv_logloss)

def train_eval_fulltrain(Xtr, ytr, Xte, yte, params, rounds):
    dtr = xgb.DMatrix(Xtr, label=ytr)
    dte = xgb.DMatrix(Xte, label=yte)

    bst = xgb.train(params=params, dtrain=dtr, num_boost_round=rounds, verbose_eval=False)

    m_tr = bst.predict(dtr, output_margin=True).astype(np.float32)
    p_tr = 1.0 / (1.0 + np.exp(-m_tr))
    train_acc = float(accuracy_score(ytr, (p_tr >= 0.5).astype(int)))

    m_te = bst.predict(dte, output_margin=True).astype(np.float32)
    p_te = 1.0 / (1.0 + np.exp(-m_te))
    test_acc = float(accuracy_score(yte, (p_te >= 0.5).astype(int)))
    test_loss = float(log_loss(yte, np.vstack([1 - p_te, p_te]).T, labels=[0, 1]))

    return train_acc, test_acc, test_loss, bst

## WeightWatcher helper

We use WeightWatcher on the PyTorch layer returned by `xgboost2ww.convert()`.

For a first pass, we run:

`watcher.analyze(randomize=True, ERG=True, plot=False)`

No `detX=True` needed for initial evaluation.

In [47]:
def ww_metrics_from_layer(layer):
    watcher = ww.WeightWatcher(model=layer)
    details_df = watcher.analyze(randomize=True, ERG=True, plot=False)  # starter: no detX
    alpha = float(details_df["alpha"].iloc[0]) if "alpha" in details_df.columns else np.nan
    traps = float(details_df["num_traps"].iloc[0]) if "rand_num_spikes" in details_df.columns else np.nan
    ERG_gap = float(details_df["ERG_gap"].iloc[0]) if "ERG_gap" in details_df.columns else np.nan
    return alpha, traps, ERG_gap

## Run the experiment (first 10 datasets)

For each dataset, we compute α/traps for **W**.

Key reproducibility detail:
- We pass `train_params=good_params` and `num_boost_round=good_rounds` into `convert()`
- That ensures the fold-training used to compute OOF increments matches the chosen model configuration.

In [ ]:
# ============================================================
# MAIN LOOP — Compute W7|W8|... ONLY using xgboost2ww.convert()
# Memory-safe version:
#   - keep sparse when possible (XGBoost supports CSR)
#   - avoid densifying big one-hot matrices
#   - use float32
#   - hard guards + cleanup
# ============================================================

import gc
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

rows = []
kept = 0

# If you ever densify, this is the real RAM killer.
# 2e8 float32 ~ 0.8GB. float64 would be ~1.6GB.
MAX_DENSE_ELEMENTS = int(2e8)

dataset_ids = enumerate_openml_dataset_ids_from_suites(SUITE_IDS)
t0 = time.time()

for did in dataset_ids:
    if kept >= TARGET_DATASETS:
        break

    loaded = load_openml_dataset_by_id(int(did))
    if loaded is None:
        continue

    X, y, name, did_loaded = loaded

    # ---- Feature guard (still useful, but not sufficient alone)
    if int(X.shape[1]) > MAX_FEATURES_GUARD:
        continue

    # -----------------------------
    # True holdout split
    # -----------------------------
    tr_idx, te_idx = train_test_split(
        np.arange(len(y)),
        test_size=TEST_SIZE,
        random_state=RNG,
        stratify=y
    )

    Xtr = X[tr_idx]
    Xte = X[te_idx]
    ytr = y[tr_idx]
    yte = y[te_idx]

    # -----------------------------
    # Keep sparse if sparse; cast to float32 if dense
    # -----------------------------
    is_sparse = hasattr(Xtr, "tocoo")  # works for scipy sparse matrices

    if not is_sparse:
        # Dense path: force 2D arrays and float32
        Xtr = np.asarray(Xtr, dtype=np.float32)
        Xte = np.asarray(Xte, dtype=np.float32)

        if Xtr.ndim != 2:
            print("SKIP: unexpected Xtr shape:", getattr(Xtr, "shape", None))
            continue
    else:
        # Sparse path: DO NOT densify for CV/training
        # Convert to CSR once (fast row slicing + DMatrix friendly)
        Xtr = Xtr.tocsr()
        Xte = Xte.tocsr()

        # OPTIONAL: you may also enforce float32 data in CSR
        # (scipy sparse supports astype efficiently)
        try:
            Xtr = Xtr.astype(np.float32)
            Xte = Xte.astype(np.float32)
        except Exception:
            pass

        # HARD guard: if someone later tries to densify, estimate cost
        dense_cost = int(Xtr.shape[0]) * int(Xtr.shape[1])
        if dense_cost > MAX_DENSE_ELEMENTS:
            # We can still train XGBoost on sparse,
            # but convert() might require dense -> likely OOM.
            # Safer to skip early unless you have a sparse-aware convert().
            print(
                f"SKIP: sparse would densify too big: "
                f"n_train={Xtr.shape[0]}, d={Xtr.shape[1]}, "
                f"elements={dense_cost:,}"
            )
            # cleanup
            del X, y, Xtr, Xte, ytr, yte
            gc.collect()
            continue

    # -----------------------------
    # Select "good" hyperparameters (train-only CV)
    # -----------------------------
    good_params, good_rounds, good_cv_logloss = pick_good_params_via_cv(
        Xtr, ytr, nfold=NFOLDS, dataset_id=int(did_loaded)
    )

    # Train final model and evaluate on holdout
    good_train_acc, good_test_acc, good_test_loss, bst = train_eval_fulltrain(
        Xtr, ytr, Xte, yte, good_params, good_rounds
    )

    if good_test_acc < MIN_GOOD_TEST_ACC:
        # cleanup
        del bst, X, y, Xtr, Xte, ytr, yte
        gc.collect()
        continue

    # ============================================================
    # Compute W using xgboost2ww.convert()
    # ============================================================
    try:
        # If convert() can accept CSR directly, great.
        # If it cannot, this will throw and we skip cleanly.
        layer_W = convert(
            model=bst,
            data=Xtr,
            labels=ytr,
            W=MATRIX,
            nfolds=NFOLDS,
            t_points=T_TRAJ,
            random_state=RNG,
            train_params=good_params,
            num_boost_round=good_rounds,
            multiclass="error",
            return_type="torch",
            verbose=False,
        )
    except Exception as e:
        print("SKIP during convert():", type(e).__name__, e)
        # cleanup
        del bst, X, y, Xtr, Xte, ytr, yte
        gc.collect()
        continue

    # WeightWatcher structural diagnostics
    alpha_W, traps_W, ERG_gap_W = ww_metrics_from_layer(layer_W)

    rows.append(dict(
        openml_id=int(did_loaded),
        dataset=name,
        n_rows_total=int(X.shape[0]),
        n_train=int(Xtr.shape[0]),
        n_test=int(Xte.shape[0]),
        n_features=int(X.shape[1]),
        rounds=int(good_rounds),
        cv_logloss=float(good_cv_logloss),
        good_train_acc=float(good_train_acc),
        good_test_acc=float(good_test_acc),
        good_test_loss=float(good_test_loss),
        alpha_W=float(alpha_W),
        traps_W=float(traps_W),
        ERG_gap_W=float(ERG_gap_W),
    ))

    kept += 1
    elapsed = (time.time() - t0) / 60.0

    print(
        f"[{kept}/{TARGET_DATASETS}] {name} (OpenML {did_loaded}) "
        f"| train/test={good_train_acc:.3f}/{good_test_acc:.3f} "
        f"| α(W)={alpha_W:.2f} traps(W)={traps_W:.1f} "
        f"| elapsed={elapsed:.1f} min",
        flush=True
    )

    # cleanup big objects each iteration
    del bst, layer_W, X, y, Xtr, Xte, ytr, yte
    gc.collect()

df_good = pd.DataFrame(rows)

print(f"\nDONE. datasets_kept={df_good['openml_id'].nunique()} rows={len(df_good)}")
df_good

[1/100] kr-vs-kp (OpenML 3) | train/test=0.995/0.986 | α(W)=2.17 traps(W)=6.0 | elapsed=1.0 min
[2/100] breast-w (OpenML 15) | train/test=0.977/0.979 | α(W)=1.90 traps(W)=6.0 | elapsed=1.1 min


## Plots

These quick plots help you sanity-check:
- α values across datasets for the W (default) matrix
- traps across datasets for the W (default) matrix
- relationship between holdout accuracy and α(W)

In [ ]:
import matplotlib.pyplot as plt

if len(df_good) == 0:
    print("No datasets kept. Try lowering MIN_GOOD_TEST_ACC.")
else:
    x = np.arange(len(df_good))

    # Alpha across datasets
    plt.figure()
    plt.plot(x, df_good[f"alpha_W"].values, label=MATRIX)
    plt.xticks(x, df_good["dataset"].values, rotation=90)
    plt.ylabel("alpha")
    plt.title("Alpha across datasets")
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Traps across datasets
    plt.figure()
    plt.plot(x, df_good[f"traps_W"].values, label=MATRIX)
    plt.xticks(x, df_good["dataset"].values, rotation=90)
    plt.ylabel("traps (rand_num_spikes)")
    plt.title("Traps across datasets")
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Holdout accuracy vs alpha(W)
    plt.figure()
    plt.scatter(df_good["good_test_acc"].values, df_good[f"alpha_W"].values)
    plt.xlabel("Holdout test accuracy")
    plt.ylabel(f"alpha W")
    plt.title(f"Holdout accuracy vs alpha{MATRIX}")
    plt.tight_layout()
    plt.show()

## Additional structural diagnostics

We now visualize the distribution of:

- alpha(W)
- traps(W)
- ERG_gap(W)

and examine the structural relationship between:

alpha(W) and ERG_gap(W)

This helps understand how scale invariance and the ERG determinant condition co-vary across models.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

if len(df_good) == 0:
    print("No results to plot.")
else:

    # -------------------------
    # Histogram: alpha_W
    # -------------------------
    plt.figure()
    plt.hist(df_good[f"alpha_W"].dropna().values, bins=30)
    plt.xlabel(f"alpha_W")
    plt.ylabel("count")
    plt.title(f"Histogram of alpha({MATRIX})")
    plt.tight_layout()
    plt.show()

    # -------------------------
    # Histogram: traps_W
    # -------------------------
    plt.figure()
    plt.hist(df_good[f"traps_W"].dropna().values, bins=30)
    plt.xlabel(f"traps_W")
    plt.ylabel("count")
    plt.title(f"Histogram of traps({MATRIX})")
    plt.tight_layout()
    plt.show()

    # -------------------------
    # Histogram: ERG_gap_W
    # -------------------------
    plt.figure()
    plt.hist(df_good[f"ERG_gap_W"].dropna().values, bins=30)
    plt.xlabel(f"ERG_gap_W7")
    plt.ylabel("count")
    plt.title(f"Histogram of ERG_gap({MATRIX})")
    plt.tight_layout()
    plt.show()

    # -------------------------
    # Scatter: alpha vs ERG_gap
    # -------------------------
    plt.figure()
    plt.scatter(
        df_good[f"alpha_W"].values,
        df_good[f"ERG_gap_W"].values
    )
    plt.xlabel(f"alpha_W")
    plt.ylabel(f"ERG_gap_W")
    plt.title(f"alpha({MATRIX}) vs ERG_gap({MATRIX})")
    plt.tight_layout()
    plt.show()

In [ ]:

# Timestamped filename so we never overwrite runs
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
RESULTS_FEATHER = os.path.join(GDRIVE_DIR, f"{MATRIX}_results_{ts}.feather")

# Convert + save
df_good = pd.DataFrame(rows)
df_good.to_feather(RESULTS_FEATHER)

print(f"Saved {len(df_good)} rows to:")
print(RESULTS_FEATHER)

df_good.head()

# RELOAD data fron Google Drive and plot

In [ ]:
# ============================================
# Load results from Google Drive + plot
# (single Colab cell)
# ============================================

from google.colab import drive
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

drive.mount("/content/drive", force_remount=False)

GDRIVE_DIR = "/content/drive/MyDrive/xgboost2ww_runs"

# Load the most recent results file by default
files = sorted(glob.glob(os.path.join(GDRIVE_DIR, f"{MATRIX}_results_*.feather")))
if not files:
    raise FileNotFoundError(f"No {MATRIX}_results_*.feather files found in {GDRIVE_DIR}")

RESULTS_FEATHER = files[-1]
print("Loading:", RESULTS_FEATHER)

df = pd.read_feather(RESULTS_FEATHER)
print("Rows:", len(df), "| Cols:", len(df.columns))
display(df.head(10))

# ---- Basic cleanup / sort
if "test_acc" in df.columns:
    df = df.sort_values("test_acc", ascending=False)
elif "good_test_acc" in df.columns:
    df = df.sort_values("good_test_acc", ascending=False)

# ---- Choose column names (supports both naming styles)
alpha_col = f"alpha_W" if f"alpha_W" in df.columns else "alpha"
traps_col = f"traps_W" if f"traps_W" in df.columns else "traps"
test_col  = "test_acc" if "test_acc" in df.columns else ("good_test_acc" if "good_test_acc" in df.columns else None)
train_col = "train_acc" if "train_acc" in df.columns else ("good_train_acc" if "good_train_acc" in df.columns else None)

# ---- Plot 1: alpha_W7 distribution
plt.figure()
plt.hist(df[alpha_col].dropna().values, bins=30)
plt.title(f"Distribution of alpha({MATRIX})")
plt.xlabel(f"alpha_W")
plt.ylabel("count")
plt.tight_layout()
plt.show()

# ---- Plot 2: traps_W distribution
plt.figure()
plt.hist(df[traps_col].dropna().values, bins=30)
plt.title(f"Distribution of traps({MATRIX})")
plt.xlabel(f"traps_W")
plt.ylabel("count")
plt.tight_layout()
plt.show()

# ---- Plot 3: alpha(W) vs test accuracy
if test_col is not None:
    plt.figure()
    plt.scatter(df[test_col].values, df[alpha_col].values)
    plt.xlabel(test_col)
    plt.ylabel(alpha_col)
    plt.title(f"alpha({MATRIX}) vs test accuracy")
    plt.tight_layout()
    plt.show()

# ---- Plot 4: train-test gap vs alpha(W) (if train exists)
if test_col is not None and train_col is not None:
    gap = df[train_col].values - df[test_col].values
    plt.figure()
    plt.scatter(gap, df[alpha_col].values)
    plt.xlabel("train - test accuracy gap")
    plt.ylabel(alpha_col)
    plt.title(f"alpha(MATRIX) vs generalization gap")
    plt.tight_layout()
    plt.show()

# ---- Quick summary table
summary_cols = [c for c in ["dataset","openml_id",train_col,test_col,alpha_col,traps_col,"rounds"] if c and c in df.columns]
print("Top 15 by test accuracy:")
display(df[summary_cols].head(15))